In [ ]:
import os
import sys
from pathlib import Path
import logging
import time
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, '..')
import src.hotspot_utils as util

In [ ]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
_LOG = logging.getLogger(__name__)

In [ ]:
client = Client(processes=False)

# Persistent Hotspot Comparison
Landgate developed a set of persistent hotspots  (v5.2) by associating recurrent hotspots with known heat sources (industrial activity). Knowledge of persistent hotspots are used here to provide a baseline for the ability of a given sensor to detect hotspots. 

Persistent hotspots, together with high resolution hotspot sources from Sentinel 2 MSI and Landsat Enhanced Thematic Mapper and Opertional Land Imager sensors are used here to validate hotspots from the AVHRR, VIIRS, MODIS and SLSTR instruments.  

In [ ]:
# This is the output directory outputs and itermediary files from this notebook examples will be stored.
outdir =  Path("/home/jovyan/s3vt_dask/s3vtdata/workdir")

# Data directory where .geojson files and presistent hotspots shape files are located.
data_dir = Path("/home/jovyan/s3vt_dask/s3vtdata/workdir")

In [ ]:
# Read persistent hotspots from a shape file in the zip folder.
persistent_hotspots_gdf = gpd.GeoDataFrame.from_file(f"zip://{data_dir.joinpath('Known non FHS - Version 5.2.zip')}/Version 5.2/known_non_FHS.shp")

In [ ]:
persistent_hotspots_gdf.head()

In [ ]:
# This is a processing parameter to load hotspots from a .geojson files within the temporal and spatial bounds provided.
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "dea_frp": None,
    "start_date": "2019-11-01",
    "end_date": "2020-10-08",
    "start_time": "21:00",
    "end_time": "3:00",
    "bbox": (147.0, -38.0, 154.0, -27.0),
    "chunks": 300,
    "outdir": outdir,
}

In [ ]:
hotspots_gdf = util.process_hotspots_gdf(**processing_parameters)

In [ ]:
persistent_hotspots_nearest_df = util.cdknearest(hotspots_gdf, persistent_hotspots_gdf)